In [1]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [4]:
column_names = ['engine_id', 'cycle', 'setting1', 'setting2', 'setting3'] + [f'sensor{i}' for i in range(1, 27)]
train_df = pd.read_csv('./dataset/train_FD001.txt', sep='\s+', header=None, names=column_names)
test_df = pd.read_csv('./dataset/test_FD001.txt', sep='\s+', header=None)
true_rul = pd.read_csv('./dataset/RUL_FD001.txt', header=None)

# Data Preprocessing
- Normalization
- Labelling of Data

In [ ]:
# Data preprocessing

# Linear Regression

# LSTM

# 

In [24]:
# Data Labelling
# RUL Column
# Label Column
# Setting of early_rul